# Generative AI Temelleri 

## 1. GenAI Temelleri ve Kavramlar

* **LLM:** İstemleri olasılıksal olarak sürdüren, milyarlarca parametreli ve geniş metinlerle eğitilmiş genel amaçlı dil modelidir.
* **Transformer Mimarisi:** Girdideki öğeler arası bağıntıları self-attention ile modelleyen, paralel hesaplamaya elverişli derin sinir ağı mimarisidir.
* **Token:** Modelin işlediği en küçük birimdir; kelime, alt kelime veya karakter olabilir ve uzunluk/maliyet bu birimlerle ölçülür.
* **Context Window:** Modelin tek seferde işleyip “hatırlayabildiği” azami token bütçesidir (girdi+çıktı) ve aşıldığında önceki bağlam dışarıda kalır.
* **System Instructions:** Modelin rolünü, sınırlarını ve üslubunu belirleyen, diğer iletilere göre önceliği yüksek üst seviye talimatlardır.
* **Hallucination:** Modelin gerçek veriye dayanmayan fakat tutarlı görünen bilgi uydurmasıdır ve RAG/ek doğrulama ile azaltılabilir.
* **Thinking (Düşünme):** Modelin cevap vermeden önce bir iç muhakeme süreci yürütmesidir. Gemini 3 ve 2.5 serisi modellerde varsayılan olarak açıktır. `thinking_budget` parametresi ile kontrol edilir: 0 = kapalı, -1 = otomatik.
* **Multimodal:** Tek bir modelin metin, görsel, ses ve video gibi farklı veri tiplerini aynı anda anlayıp işleyebilmesidir.

## 2. LLM-Based Uygulama Geliştirme: Temel Enstrümanlar

### 2.1. Model Seçimi 

- **API based modeller:** Claude, OpenAI, Google (Gemini) vb.
- **Local Modeller:** Deepseek, Google (Gemma), Kumru vb.
- **Model Büyüklüğü:** 2B, 7B, 40B 
- **Multimodal:** Text, image, audio, video vb.
 
### 2.2. Prompt (İstek/Talimat)

Prompt, modele ne yapmasını istediğinizi söyleyen metindir.

- **Prompt Anatomisi**:
```
[System Instruction] + [Context] + [Task] + [Format] + [Examples]
```

- **Prompt Kalitesi = Output Kalitesi**

### 2.3. Model Parametreleri

| Parametre | Açıklama | Tipik Aralık |
|-----------|----------|-------------|
| **temperature** | Örnekleme rastgeleliği. Düşük = tutarlı, yüksek = yaratıcı | 0.0 - 2.0 |
| **max_output_tokens** | Tek yanıttaki azami token sayısı | 1 - 65536 |
| **top_p** | Olasılık kütlesinden çekirdek örnekleme eşiği | 0.0 - 1.0 |
| **top_k** | En olası k aday token arasından seçim | 1 - 40+ |
| **thinking_budget** | Düşünme token bütçesi (0=kapalı, -1=otomatik) | 0 - 24576 |


### 2.4. Ek Enstrümanlar

- **Safety Settings**: Zararlı içerik filtreleme
- **Function Calling**: External tool'lara erişim
- **Response Schema**: Structured output (JSON)
- **Thinking Config**: Modelin düşünme davranışını kontrol etme
- **Code Execution**: Modelin Python kodu çalıştırabilmesi
- **vb.**

## 3. Gemini Modelleri - [Gemini Docs](https://ai.google.dev/gemini-api/docs?hl=tr)

| Özellik | Gemini 3 Flash Preview |
|---------|----------------------|
| Context Window | 1M input token |
| Max Output | 65.5K token |
| Thinking | Varsayılan açık |
| Multimodal | Text, image, audio, video |



## 4. Gemini API Key Alma ve Kurulum

### 4.1. API Key Alma

1. **Google AI Studio'ya gidin**: [https://aistudio.google.com/](https://aistudio.google.com/)
2. Google hesabınızla giriş yapın
3. Sol menüden **"Get API Key"** seçeneğine tıklayın
4. **"Create API Key"** butonuna basın
5. Yeni bir API key oluşturun veya mevcut bir projeye ekleyin
6. API key'inizi kopyalayın ve güvenli bir yerde saklayın

**Güvenlik Uyarısı**: API key'inizi asla public repository'lere commit etmeyin! [(.gitignore)](https://github.com/github/gitignore/blob/main/Python.gitignore)



### 4.2. Python SDK Kurulumu

```bash
# Google Gen AI SDK'sını yükleyin
pip install -q -U google-genai

# Alternatif: requirements.txt dosyasına ekleyin
"google-genai>=1.0.0" 
pip install -r requirements.txt
```

In [ ]:
# kurulum
#pip install -q -U google-genai

### 4.3. İlk Yapılandırma

In [ ]:
import os
from dotenv import load_dotenv
from google import genai
from google.genai import types

load_dotenv()

api_key = os.getenv('GEMINI_API_KEY')
client = genai.Client(api_key=api_key)

#MODEL = 'gemini-3-flash-preview'
MODEL = 'gemini-2.5-flash'

# Thinking off config - disable model's internal reasoning
THINK_OFF = types.ThinkingConfig(thinking_budget=0)

#Extract only text parts from response, skipping thought_signature
def get_text(response):
    return "".join(
        part.text for part in response.candidates[0].content.parts
        if part.text and not part.thought
    )

In [ ]:
# check model info
model_info = client.models.get(model=MODEL)
model_info

In [ ]:
# thinking ON - model responds with internal reasoning (default behavior)
response = client.models.generate_content(
    model=MODEL,
    contents='Selamlar nasılsın, şuan bir eğitimdeyiz herkese selam söyle.',
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(include_thoughts=True)
    )
)
# show thinking parts
for part in response.candidates[0].content.parts:
    if part.thought:
        print("[THINKING]", part.text, "...")
    else:
        print("[RESPONSE]", part.text)

In [ ]:
# thinking OFF - model responds directly without reasoning
response = client.models.generate_content(
    model=MODEL,
    contents='Selamlar nasılsın, şuan bir eğitimdeyiz herkese selam söyle.',
    config=types.GenerateContentConfig(thinking_config=THINK_OFF)
)
print(get_text(response))


## Parameters

### Temperature

In [ ]:
# default temperature - varied outputs each time
outputs = []
prompt = "Türkiye Yapay Zeka Topluluğu hakkında sadece 1 cümlelik bilgi ver."
config = types.GenerateContentConfig(temperature=2, thinking_config=THINK_OFF)
for i in range(5):
    response = client.models.generate_content(model=MODEL, contents=prompt, config=config)
    outputs.append(get_text(response))
for index, sentence in enumerate(outputs, start=1):
    print(f"{index}. {sentence}")

In [ ]:
# temperature=0 - deterministic, same output each time
new_outputs = []
low_temp_config = types.GenerateContentConfig(temperature=0, thinking_config=THINK_OFF)
for i in range(5):
    response = client.models.generate_content(
        model=MODEL, contents=prompt, config=low_temp_config
    )
    new_outputs.append(get_text(response))
for index, sentence in enumerate(new_outputs, start=1):
    print(f"{index}. {sentence}")

## Max Output Length


In [ ]:
# no max_output_tokens limit - model decides output length
prompt = "Türkiye Yapay Zeka Topluluğu hakkında bilgi verir misiniz?"
response = client.models.generate_content(
    model=MODEL, contents=prompt,
    config=types.GenerateContentConfig(thinking_config=THINK_OFF)
)
print(get_text(response))

In [ ]:
# max_output_tokens=200 - limits response length
prompt = "Türkiye Yapay zeka Topluluğu hakkında bilgi verir misiniz?"
response = client.models.generate_content(
    model=MODEL,
    contents=prompt,
    config=types.GenerateContentConfig(max_output_tokens=200, thinking_config=THINK_OFF)
)
print(get_text(response))

## Token Count

In [ ]:
poem_prompt = "Bilgisayarlar hakkında az bilinen 5 bilgi ver."
token_config = types.GenerateContentConfig(temperature=0.5, thinking_config=THINK_OFF)

In [ ]:
response = client.models.generate_content(
    model=MODEL, contents=poem_prompt, config=token_config
)
print(get_text(response))

In [ ]:
prompt_token_count = client.models.count_tokens(model=MODEL, contents=poem_prompt)
output_token_count = client.models.count_tokens(model=MODEL, contents=get_text(response))
print(f'Tokens in prompt: {prompt_token_count.total_tokens}')
print(f'Estimated tokens in output: {output_token_count.total_tokens}')

## Prompt Teknikleri

### Açık ve Net Talimatlar

Prompt kalitesi doğrudan çıktı kalitesini belirler. Aşağıda kötü ve iyi prompt örnekleri:

**Kötü Prompt:** Belirsiz, model ne istediğinizi tahmin etmek zorunda
```python
prompt = "Python hakkında bir şeyler söyle."
```

**İyi Prompt:** Spesifik görev, format ve kapsam belirli
```python
prompt = """
Python programlama dilinin aşağıdaki özelliklerini açıkla:
1. Liste comprehension nedir ve nasıl kullanılır?
2. Decorator'lar ne işe yarar?
3. Generator fonksiyonları neden kullanılır?

Her madde için bir kod örneği ver.
"""
```


### System Prompt

System instruction, modele bir kimlik ve davranış kuralları verir. 

In [ ]:
response_with_sys = client.models.generate_content(
    model=MODEL,
    contents="Yapay zeka nedir?",
    config=types.GenerateContentConfig(
        thinking_config=THINK_OFF,
        system_instruction="""Sen Türkiye Yapay Zeka Topluluğunun akil küpü yapay zeka asistanisin. 
        Kullanıcılara sadece Türkiye yapay zeka dışında bir bilgi verme, sadece bu bilgilendirmeyi yap."""
    )
)
print("=== System Instruction VAR ===")
print(get_text(response_with_sys))

### Role-Based Prompting (Rol Tanımlama)

Aynı soruyu farklı rollerle sorarak çıktının nasıl değiştiğini görelim.

In [ ]:
question = "Bir e-ticaret sitesinde ürün arama performansı düşük. Ne yapmalıyız?"

roles = {
    "Junior Developer": "Sen 1 yıllık deneyime sahip junior bir yazılımcısın. Kısa ve basit öneriler ver.",
    "Senior Architect": "Sen 15 yıllık deneyimli bir yazılım mimarısın. Sistem tasarımı perspektifinden yanıt ver.",
    "Product Manager": "Sen bir ürün yöneticisisin. Kullanıcı deneyimi ve iş metrikleri perspektifinden yanıt ver."
}

role_list = list(roles.items())

def get_role_response(role_index):
    role_name, instruction = role_list[role_index]
    response = client.models.generate_content(
        model=MODEL, contents=question,
        config=types.GenerateContentConfig(
            thinking_config=THINK_OFF,
            system_instruction=instruction,
            max_output_tokens=500
        )
    )
    print(f"--- {role_name} ---")
    print(get_text(response))

In [ ]:
# Junior Developer
get_role_response(0)

In [ ]:
# Senior Architect
get_role_response(1)

In [ ]:
# Product Manager
get_role_response(2)

### Structured Output (JSON)

Modelden yapılandırılmış veri çıkarma. Aşağıda bir iş ilanından bilgileri JSON formatında çıkarıyoruz.

In [ ]:
import json

job_posting = """
Pozisyon: Senior Data Scientist
Şirket: TrAI Yazılım A.Ş.
Lokasyon: İstanbul (Hibrit - Haftada 2 gün ofis)
Maaş Aralığı: 150.000 - 200.000 TL
Gereksinimler:
- Python, SQL ve Spark deneyimi (en az 4 yıl)
- Makine öğrenimi model geliştirme tecrübesi
- İyi derecede İngilizce
- Üniversite mezunu (Bilgisayar Mühendisliği, İstatistik veya ilgili alan)
Arti Nitelikler: MLOps, Docker, AWS deneyimi
"""

prompt = f"""Aşağıdaki iş ilanından bilgileri çıkar ve SADECE JSON formatında döndür:

{job_posting}

JSON şeması:
{{
  "position": "string",
  "company": "string",
  "location": "string",
  "work_model": "string",
  "salary_min": number,
  "salary_max": number,
  "min_experience_years": number,
  "required_skills": ["string"],
  "nice_to_have": ["string"],
  "education": "string"
}}
"""

response = client.models.generate_content(
    model=MODEL, contents=prompt,
    config=types.GenerateContentConfig(thinking_config=THINK_OFF)
)
result = get_text(response)
print(result)

## Zero-Shot Prompting

Hiç örnek vermeden, sadece görev tanımıyla model yönlendirme. Aşağıda bir müşteri destek mesajını otomatik kategorilere ayırıyoruz.

In [ ]:
# Zero-shot: Hiç örnek vermeden kategori belirleme
tickets = [
    "Siparişim 5 gündür gelmedi, kargo nerede? Çok sinirli oldum artık!",
    "Ürünlerinize renk filtresi ekleseniz çok iyi olur, aramayı kolaylaştırır.",
    "Geçen hafta aldığım laptop hakkında garanti süresini öğrenmek istiyorum.",
    "Harika bir alışveriş deneyimiydi, müşteri hizmetleri çok ilgiliydi, teşekkürler!"
]

prompt_template = """Aşağıdaki müşteri mesajını analiz et.
Kategori: şikayet / öneri / bilgi_talebi / teşekkür
Aciliyet: düşük / orta / yüksek
Sadece bu iki bilgiyi ver, açıklama ekleme.

Mesaj: "{ticket}"
"""

for ticket in tickets:
    response = client.models.generate_content(
        model=MODEL,
        contents=prompt_template.format(ticket=ticket),
        config=types.GenerateContentConfig(thinking_config=THINK_OFF, max_output_tokens=50)
    )
    print(f"Mesaj: {ticket[:60]}...")
    print(get_text(response))
    print()

## Few-Shot Learning (Örneklerle Öğretme)

Birkaç örnek vererek modele custom bir format/davranış öğretme. Zero-shot'tan farkı: model görmediği bir format veya kural setini örneklerden öğrenir. Aşağıda yapılandırılmamış e-posta metninden veri çıkarmayı öğretiyoruz.

In [ ]:
# Few-shot: Örneklerle custom format öğretme
prompt = """E-posta metninden yapılandırılmış veri çıkar.

--- ÖRNEK 1 ---
E-posta: "Merhaba, ben Ayşe Kara. 15 Ocak'ta sipariş ettiğim #ORD-4521 numaralı ürün hasarlı geldi. Faturamı da bulamıyorum. İade yapmak istiyorum. Tel: 0532 111 22 33"
Çıktı:
- Müşteri: Ayşe Kara
- Sipariş No: #ORD-4521
- Sorun: Hasarlı ürün
- Talep: İade
- İletişim: 0532 111 22 33

--- ÖRNEK 2 ---
E-posta: "Mehmet Yıldız yazıyorum. Dün aldığım monitörün (#ORD-7890) ölü pikseli var, değişim talep ediyorum. Mail: mehmet@email.com"
Çıktı:
- Müşteri: Mehmet Yıldız
- Sipariş No: #ORD-7890
- Sorun: Ölü piksel
- Talep: Değişim
- İletişim: mehmet@email.com

--- ŞİMDİ SEN ÇÖZ ---
E-posta: "Selam, Zeynep Demir. Geçen hafta sipariş verdiğim kulaklık (#ORD-3156) kutusunda şarj kablosu eksik, tamamlayabilir misiniz? Bana 0555 987 65 43 ten ulaşabilirsiniz."
Çıktı:
"""

response = client.models.generate_content(
    model=MODEL, contents=prompt,
    config=types.GenerateContentConfig(thinking_config=THINK_OFF, max_output_tokens=150)
)
print(get_text(response))

## Chain-of-Thought (Düşünce Zinciri)

Modelden adım adım muhakeme yapmasını isteyerek daha doğru sonuçlar elde etme. Basit hesaplamalar yerine, birden fazla kriteri tartmayı gerektiren karmaşık bir teknik karar problemi verelim.

In [ ]:
prompt = """
Adım adım düşünerek aşağıdaki teknik kararı analiz et:

SENARYO:
Bir startup, günlük 50.000 kullanıcının etkileşimde bulunduğu bir sosyal medya uygulaması geliştiriyor. 
Kullanıcılar post paylaşıyor, yorum yapıyor ve birbirini takip ediyor.
Gelecek 6 ayda 500.000 kullanıcıya ölçeklenmeyi planlıyorlar.

SORU: Veritabanı olarak PostgreSQL mu yoksa MongoDB mi tercih etmeliler?

ANALİZ FORMATI:
1. Veri yapısını analiz et (ilişkisel mi, döküman tabanlı mı?)
2. Her seçenek için avantaj ve dezavantajları listele
3. Ölçeklenme gereksinimlerini değerlendir
4. Nihai önerini gerekçesiyle sun
"""

response = client.models.generate_content(
    model=MODEL, contents=prompt,
    config=types.GenerateContentConfig(thinking_config=THINK_OFF, max_output_tokens=800)
)
print(get_text(response))

## Bağlamı koruyarak chat in devam etmesi

Gemini SDK `chats.create()` ile başlattığınız oturumda **conversation memory** otomatik çalışır: her `send_message` çağrısında önceki tüm mesajlar modele gönderilir, böylece model bağlamı korur. Aşağıda önce bağlam olmadan ne olacağını, sonra bağlamlı sohbeti ve geçmişi nasıl inceleyeceğinizi görüyorsunuz.

In [ ]:
# Chat oturumu başlat
chat = client.chats.create(
    model=MODEL,
    config=types.GenerateContentConfig(thinking_config=THINK_OFF, max_output_tokens=800)
)

# İlk mesaj
response1 = chat.send_message("Rust öğrenmek istiyorum nereden başlayabilirim?")
print("Bot:", get_text(response1))

In [ ]:
# İkinci mesaj (bağlam korunur)
response2 = chat.send_message("Peki, hangi IDE'yi önerirsin?")
print("Bot:", get_text(response2))

In [ ]:
# Üçüncü mesaj
response3 = chat.send_message("Bu programlama dilinin ana olayı nedir?")
print("Bot:", get_text(response3))

In [ ]:
# Sohbet geçmişini görüntüle
print("\n--- Sohbet Geçmişi ---")
for message in chat.get_history():
    print(f"{message.role}: {message.parts[0].text}\n")

## Hepsini bir araya getirelim

In [ ]:
import os
from dotenv import load_dotenv
from google import genai
from google.genai import types
import gradio as gr

load_dotenv()
gr_client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))


def get_text(response):
    return "".join(
        part.text for part in response.candidates[0].content.parts
        if part.text and not part.thought
    )

CHAT_CONFIG = types.GenerateContentConfig(
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    max_output_tokens=2048,
    thinking_config=types.ThinkingConfig(thinking_budget=0),
    system_instruction="""Sen KrediPusula projesinin kredi risk danışmanısın. KrediPusula, kullanıcıların kredi uygunluğunu analiz eden, kişiselleştirilmiş kredi önerileri sunan akıllı kredi danışmanlık platformudur.

Görevin:
- Kredi riski, kredi skoru ve kredi uygunluğu hakkında bilgi vermek
- Kullanıcıları gelir, yaş, meslek, tasarruf durumu gibi faktörlerin kredi başvurusuna etkisi konusunda yönlendirmek
- Kredi hesaplama, faiz oranları ve taksit seçenekleri hakkında genel bilgi sunmak
- Başvuru süreci, gerekli belgeler ve platformun nasıl kullanılacağı konusunda yardımcı olmak

Kurallar:
- Yalnızca kredi, risk ve KrediPusula platformu ile ilgili sorulara yanıt ver
- Kesin onay/red kararı verme; nihai karar model ve banka süreçlerine aittir
- Türkçe yanıt ver, kısa ve anlaşılır ol
- Bilmediğin konularda tahmin yürütme, platformdaki başvuru formunu öner"""
)


def chat_function(message, history):
    if not message or message.strip() == "":
        return "Lütfen bir mesaj yazın."

    chat_history = []
    for human, assistant in history:
        if human and assistant:
            chat_history.append(
                types.Content(role="user", parts=[types.Part.from_text(text=human)])
            )
            chat_history.append(
                types.Content(role="model", parts=[types.Part.from_text(text=assistant)])
            )

    chat = gr_client.chats.create(
        model='gemini-3-flash-preview', config=CHAT_CONFIG, history=chat_history
    )
    response = chat.send_message(message.strip())
    return get_text(response)


demo = gr.ChatInterface(
    fn=chat_function,
    title="KrediPusula - Kredi Risk Danışmanı",
    description="Kredi uygunluğu, risk skoru ve kredi başvurusu hakkında sorularınızı sorun",
    examples=[
        "Kredi risk skoru nedir, nasıl hesaplanır?",
        "Gelirim düşük, kredi alabilir miyim?",
        "Kredi başvurusu için hangi belgeler gerekli?",
        "Tasarruf hesabım yok, bu kredi onayımı etkiler mi?"
    ],
    theme="soft"
)


if __name__ == "__main__":
    demo.launch(share=True)
